In [34]:
import pandas as pd


In [35]:
df = pd.read_csv("applications.csv")
df[:5]

,app_name,file_path,category
0,Visual Studio,C:\Program Files (x86)\Microsoft Visual Studio...,work
1,Eclipse,C:\Users\username\eclipse\java-2021-06\eclipse...,work
2,IntelliJ IDEA,C:\Program Files\JetBrains\IntelliJ IDEA 2021....,work
3,PyCharm,C:\Program Files\JetBrains\PyCharm 2021.1.1\bi...,work
4,WebStorm,C:\Program Files\JetBrains\WebStorm 2021.1.1\b...,work


In [36]:
df.drop_duplicates(inplace=True)

In [37]:
df.shape

(526, 3)

In [38]:
df.to_csv("applications.csv", index=False)

In [39]:
df = pd.read_csv("applications.csv")
df[:5]

,app_name,file_path,category
0,Visual Studio,C:\Program Files (x86)\Microsoft Visual Studio...,work
1,Eclipse,C:\Users\username\eclipse\java-2021-06\eclipse...,work
2,IntelliJ IDEA,C:\Program Files\JetBrains\IntelliJ IDEA 2021....,work
3,PyCharm,C:\Program Files\JetBrains\PyCharm 2021.1.1\bi...,work
4,WebStorm,C:\Program Files\JetBrains\WebStorm 2021.1.1\b...,work


In [40]:
len(df)

526

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
df = pd.read_csv('applications.csv')

# Clean and preprocess the data
df['app_name'] = df['app_name'].str.lower().str.strip()
df['file_path'] = df['file_path'].str.lower().str.strip()

# Extract additional features from the 'file_path' column
df['dir_depth'] = df['file_path'].str.count(r'\\') + 1  # Adjusted for Windows paths
df['domain'] = df['file_path'].str.split(r'\\').str[2]  # Assuming domain is the third part

# Initialize CountVectorizer for each column
vectorizer_app = CountVectorizer()
vectorizer_path = CountVectorizer()

# Fit and transform the 'app_name' and 'file_path' columns separately
X_app = vectorizer_app.fit_transform(df['app_name'])
X_path = vectorizer_path.fit_transform(df['file_path'])

# Convert the matrices to arrays
X_app_array = X_app.toarray()
X_path_array = X_path.toarray()

# Create DataFrames from the arrays
df_app = pd.DataFrame(X_app_array, columns=vectorizer_app.get_feature_names_out())
df_path = pd.DataFrame(X_path_array, columns=vectorizer_path.get_feature_names_out())

# One-hot encode the 'domain' column
encoder = OneHotEncoder(sparse_output=False)
domain_encoded = encoder.fit_transform(df[['domain']])
df_domain = pd.DataFrame(domain_encoded, columns=encoder.get_feature_names_out(['domain']))

# Concatenate the DataFrames
X_combined = pd.concat([df_app, df_path, df[['dir_depth']].reset_index(drop=True), df_domain], axis=1)

# Prepare the target variable
y = df['category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred, average='weighted'))
print('Recall:', recall_score(y_test, y_pred, average='weighted'))
print('F1-score:', f1_score(y_test, y_pred, average='weighted'))


Accuracy: 0.660377358490566
Precision: 0.7096800656275636
Recall: 0.660377358490566
F1-score: 0.6032770605759682


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
df = pd.read_csv('applications.csv')

# Clean and preprocess the data
df['app_name'] = df['file_path'].str.lower().str.strip().str.split('\\').str[-1]
# print(df['app_name'][:2])
# Initialize CountVectorizer for 'app_name'
vectorizer_app = CountVectorizer()

# Fit and transform the 'app_name' column
X_app = vectorizer_app.fit_transform(df['app_name'])

# Convert the matrix to array
X_app_array = X_app.toarray()

# Create DataFrame from the array
df_app = pd.DataFrame(X_app_array, columns=vectorizer_app.get_feature_names_out())

# Prepare the target variable
y = df['category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_app, y, test_size=0.2, random_state=42)

# Train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

# New application name
new_app_name = ["pdr.exe"]

# Transform the new app name using the fitted vectorizer
new_app_vectorized = vectorizer_app.transform(new_app_name)

# Convert the sparse matrix to array
new_app_array = new_app_vectorized.toarray()

# Predict the category
predicted_category = model.predict(new_app_array)

print("Predicted Category:", predicted_category[0])


Accuracy: 0.6037735849056604
Precision: 0.6085868309587986
Recall: 0.6037735849056604
F1-score: 0.5093725476815996
Predicted Category: entertainment


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [12]:
df

,app_name,file_path,category
0,devenv.exe,C:\Program Files (x86)\Microsoft Visual Studio...,work
1,eclipse.exe,C:\Users\username\eclipse\java-2021-06\eclipse...,work
2,idea64.exe,C:\Program Files\JetBrains\IntelliJ IDEA 2021....,work
3,pycharm64.exe,C:\Program Files\JetBrains\PyCharm 2021.1.1\bi...,work
4,webstorm64.exe,C:\Program Files\JetBrains\WebStorm 2021.1.1\b...,work
...,...,...,...
521,icecat.exe,C:\Program Files\GNU IceCat\icecat.exe,personal
522,konqueror.exe,C:\Program Files\Konqueror\konqueror.exe,personal
523,navigator.exe,C:\Program Files\Netscape\Navigator\navigator.exe,personal
524,duckduckgo.exe,C:\Program Files\DuckDuckGo Privacy Browser\du...,personal


In [ ]:
import pyautogui
import time
import pyperclip

# Function to bring the browser to focus by its title
def focus_browser_window(title):
    # Locate the center of the window with the specified title
    window = pyautogui.getWindowsWithTitle(title)
    if window:
        window[0].activate()
        time.sleep(0.3)  # Adjusted wait time

# Function to simulate 'Ctrl+L' and 'Ctrl+C' to copy the current URL
def copy_url_from_browser():
    # Select the URL in the address bar
    pyautogui.hotkey('ctrl', 'l')
    time.sleep(0.2)  # Adjusted wait time

    # Copy the URL to clipboard
    pyautogui.hotkey('ctrl', 'c')
    time.sleep(0.2)  # Adjusted wait time

    # Return the copied URL
    return pyperclip.paste()

# Function to switch to the next tab in most browsers using 'Ctrl+Tab'
def switch_to_next_tab():
    pyautogui.hotkey('ctrl', 'tab')
    time.sleep(0.3)  # Adjusted wait time

# Main logic
if __name__ == "__main__":
    browser_title = input("Enter a part of your browser's title (e.g., 'Google Chrome'): ")
    number_of_tabs = int(input("Enter the number of tabs open in your browser: "))
    
    focus_browser_window(browser_title)
    
    try:
        for _ in range(number_of_tabs):
            url = copy_url_from_browser()
            print(f"URL of current tab: {url}")
            switch_to_next_tab()
    except KeyboardInterrupt:
        print("\nMonitoring stopped by user.")


Enter a part of your browser's title (e.g., 'Google Chrome'): google


In [11]:
import pyautogui
import time
import pyperclip
import pygetwindow as gw
import win32gui

# Function to list all open windows and print their titles
def list_open_windows():
    windows = gw.getAllTitles()
    for window in windows:
        if window.strip():  # Skip empty titles
            print(window)

# Function to bring the browser to focus by its title
def focus_browser_window(title):
    windows = [w for w in gw.getWindowsWithTitle(title) if title.lower() in w.title.lower()]
    if windows:
        windows[0].activate()
        time.sleep(1)  # Wait for the browser to come into focus
    else:
        print(f"No window found with title containing: {title}")

# Function to simulate 'Ctrl+L' and 'Ctrl+C' to copy the current URL
def copy_url_from_browser():
    pyautogui.hotkey('ctrl', 'l')
    time.sleep(0.1)  # Wait for the address bar to be selected
    pyautogui.hotkey('ctrl', 'c')
    time.sleep(0.1)  # Wait for the copy command to complete
    return pyperclip.paste()

# Function to monitor for new tabs
def monitor_tabs(browser_title, check_interval=2):
    copied_urls = set()  # Set to store copied URLs
    current_windows = set()

    while True:
        try:
            # Get a list of windows that match the browser title
            windows = [w for w in gw.getWindowsWithTitle(browser_title) if browser_title.lower() in w.title.lower()]
            if not windows:
                print(f"No window found with title containing: {browser_title}")
                list_open_windows()  # List all open windows for debugging
                time.sleep(check_interval)
                continue

            # Get current window titles
            new_windows = {w.title for w in windows}
            new_tabs = new_windows - current_windows

            for tab in new_tabs:
                print(f"New tab detected: {tab}")
                focus_browser_window(tab)
                new_url = copy_url_from_browser()

                if new_url not in copied_urls:
                    copied_urls.add(new_url)
                    print(f"URL copied: {new_url}")

            current_windows = new_windows
            time.sleep(check_interval)  # Delay before checking again

        except KeyboardInterrupt:
            print("Monitoring stopped by user.")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Main logic
if __name__ == "__main__":
    browser_title = input("Enter a part of your browser's title (e.g., 'Google Chrome'): ")
    if not browser_title.strip():
        print("You must enter a part of the browser's title to continue.")
    else:
        monitor_tabs(browser_title)


Enter a part of your browser's title (e.g., 'Google Chrome'): google
New tab detected: Untitled - Jupyter Notebook - Google Chrome
URL copied: http://localhost:8888/notebooks/Untitled.ipynb
New tab detected: Home - Google Chrome
URL copied: https://chatgpt.com/c/439991ba-2412-4fb4-8174-b79b713d2d84
New tab detected: ChatGPT - Google Chrome
New tab detected: Untitled - Jupyter Notebook - Google Chrome
New tab detected: Home - Google Chrome
New tab detected: ChatGPT - Google Chrome
New tab detected: New Tab - Google Chrome
New tab detected: mail.google.com/mail/u/0/?tab=rm&ogbl - Google Chrome
URL copied: https://mail.google.com/mail/u/0/?tab=rm&ogbl
New tab detected: Untitled - Jupyter Notebook - Google Chrome
New tab detected: Inbox (89) - tarunutla1234@gmail.com - Gmail - Google Chrome
New tab detected: Untitled - Jupyter Notebook - Google Chrome
Monitoring stopped by user.


In [30]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import joblib
import numpy as np
from collections import Counter



In [32]:
# Sample data (replace with your actual dataset)
df = pd.read_csv("C:\\Users\\Admin\\OneDrive\\Desktop\\tmachine\\smartAgile\\browsertasks.csv")


In [33]:
df.duplicated().sum()

np.int64(691)

In [34]:
# # Data cleaning (assuming 'keyword' and 'category' are the relevant columns)
df.dropna(subset=['keyword', 'category'], inplace=True)  # Drop rows with NaN values
df.drop_duplicates(subset=['keyword'], inplace=True)  # Drop duplicate keywords

In [35]:
len(df)

5030

In [36]:
# Text preprocessing
stop_words = set(stopwords.words('english'))  # Expand this list based on your dataset's context
lemmatizer = WordNetLemmatizer()

import re

def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    
    # Removing punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing stopwords and punctuation
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Removing numbers (if they're not relevant)
    tokens = [token for token in tokens if not token.isdigit()]
    
    # Removing extra whitespace
    text = " ".join(tokens).strip()
    
    return text


In [37]:

df['clean_keyword'] = df['keyword'].apply(preprocess_text)

# Vectorization using TF-IDF
vectorizer = TfidfVectorizer()  # Removed max_features to let feature selection happen in GridSearchCV

# Initialize SVM classifier within a pipeline
pipeline = Pipeline([
    ('tfidf', vectorizer),
    ('svm', SVC(kernel='linear', random_state=42))  # Linear kernel is chosen for simplicity
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['clean_keyword'], df['category'], test_size=0.3, random_state=42)

# Hyperparameter tuning using GridSearchCV with expanded parameter grid and cross-validation
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto'],
    'tfidf__max_features': [500, 1000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)]# None means all features are used
}
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
best_pipeline = grid_search.best_estimator_

# Predict on the test set using the best pipeline
y_pred = best_pipeline.predict(X_test)

# Evaluate the model with additional metrics
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix and Error Analysis
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)


# Save the trained pipeline (includes both vectorizer and model)
joblib.dump(best_pipeline, 'svm_pipeline.pkl')

# Load the saved pipeline
loaded_pipeline = joblib.load('svm_pipeline.pkl')

# Example: Predicting new data using the loaded pipeline
new_data = ["New project planning techniques", "video songs"]
predictions = loaded_pipeline.predict(new_data)
print("\nPredictions for new data:")
for text, prediction in zip(new_data, predictions):
    print(f"{text} --> {prediction}")


C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Best Parameters: {'svm__C': 1, 'svm__gamma': 'scale', 'tfidf__max_features': None, 'tfidf__ngram_range': (1, 1)}

Accuracy: 0.8601722995361166

Classification Report:
                         precision    recall  f1-score   support

   Educational-related       0.88      0.57      0.69        67
 Entertainment-related       0.72      0.64      0.68        59
              Personal       0.95      0.99      0.97        92
     Education-related       0.95      0.49      0.65        43
   Educational-related       0.80      0.89      0.84       264
 Entertainment related       0.91      0.86      0.88       181
 Entertainment-related       0.87      0.90      0.88       373
       Finance-related       0.89      0.80      0.84        60
        Gaming-related       0.87      0.89      0.88        37
                Health       0.83      0.94      0.88        16
              Personal       0.00      0.00      0.00         1
          Work-related       0.86      0.91      0.89       316

C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo


Predictions for new data:
New project planning techniques --> Work-related
video songs --> Entertainment-related


In [49]:
# X_new = vectorizer.transform()
loaded_pipeline = joblib.load('svm_pipeline.pkl')
loaded_pipeline.predict(["Dhanush's Jabardasth Telugu Comedy Back 2 Back Comedy Scenes || Latest Telugu Comedy 2016 - YouTube - Brave"])

array(['Entertainment-related'], dtype=object)

In [47]:
task="Dhanush's Jabardasth Telugu Comedy Back 2 Back Comedy Scenes || Latest Telugu Comedy 2016 - YouTube - Brave"
prediction_input = pd.DataFrame([task],columns=['Task'])
print(prediction_input)

                                                Task
0  Dhanush's Jabardasth Telugu Comedy Back 2 Back...


In [51]:
loaded_pipeline.predict(prediction_input)[0]

'Work-related'

In [26]:
import pandas as pd
import re
import spacy
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# Ensure necessary downloads
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')

# Initialize NLP tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
sid = SentimentIntensityAnalyzer()
nlp = spacy.load('en_core_web_sm')

# Example dataset
df = pd.read_csv("C:\\Users\\Admin\\OneDrive\\Desktop\\tmachine\\smartAgile\\browsertasks.csv")

# Adding custom stop words based on context
additional_stopwords = {'project', 'planning', 'techniques'}
stop_words.update(additional_stopwords)

def handle_negations(text):
    if sid.polarity_scores(text)['neg'] > 0.5:
        return 'negated ' + text
    return text

def spacy_tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc if not token.is_stop and not token.is_punct]

def stem_and_lemmatize(tokens):
    return [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens]

def advanced_preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = handle_negations(text)  # Handle negations
    tokens = spacy_tokenize(text)  # Advanced tokenization
    tokens = stem_and_lemmatize(tokens)  # Stemming and lemmatization
    tokens = [token for token in tokens if token not in stop_words and not token.isdigit()]  # Remove stop words and digits
    text = " ".join(tokens).strip()  # Join tokens into a clean string
    return text

# Apply advanced preprocessing
df['clean_keyword'] = df['keyword'].apply(advanced_preprocess_text)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df['clean_keyword'], df['category'], test_size=0.3, random_state=42)

# Define the TF-IDF vectorizer and the SVM classifier within a pipeline
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
pipeline = Pipeline([
    ('tfidf', vectorizer),
    ('svm', SVC(kernel='linear', random_state=42))
])

# Define the hyperparameter grid for tuning
param_grid = {
    'svm__C': [0.1, 1, 10, 100, 1000],
    'svm__gamma': ['scale', 'auto', 0.1, 0.01],
    'tfidf__max_features': [500, 1000, 5000, None],
    'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)]
}

# Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Output the best parameters and model
print("Best Parameters:", grid_search.best_params_)
best_pipeline = grid_search.best_estimator_

# Evaluate the model on the test set
y_pred = best_pipeline.predict(X_test)
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)

# Save the trained model pipeline
joblib.dump(best_pipeline, 'enhanced_svm_pipeline.pkl')

# Load the saved model pipeline
loaded_pipeline = joblib.load('enhanced_svm_pipeline.pkl')

# Predicting new data using the loaded pipeline
new_data = ["New project planning techniques", "Enjoyable video songs"]
predictions = loaded_pipeline.predict(new_data)
print("\nPredictions for new data:")
for text, prediction in zip(new_data, predictions):
    print(f"{text} --> {prediction}")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

Note: you may need to restart the kernel to use updated packages.Collecting spacy
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached langcodes-3.4.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.2.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached cloudpathlib-0.18.1-py3-none-any.whl.metadata (14 kB)
  Using cached smart_open-7.0.4-py3-none-any